In [1]:
%env DGLBACKEND=tensorflow
import dgl

env: DGLBACKEND=tensorflow


2022-04-26 01:23:49.634261: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/manisb/opt/anaconda3/envs/ml-project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = dgl.data.CiteseerGraphDataset()

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
graph = dataset[0]
adj = graph.adjacency_matrix()

In [4]:
features = graph.ndata['feat']
labels = graph.ndata['label']
train_mask = graph.ndata['train_mask']
test_mask = graph.ndata['test_mask']
val_mask = graph.ndata['val_mask']

In [5]:
import tensorflow as tf

class GCN_Layer(tf.keras.layers.Layer):
    def __init__(self, adjacency_matrix, input_dim, output_dim):
        super(GCN_Layer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        a_i = tf.add(
            tf.sparse.to_dense(tf.sparse.reorder(adjacency_matrix)), 
            tf.eye(adjacency_matrix.shape[0]))
        self.a_cap = tf.linalg.normalize(a_i, axis=0)[0]
        self.w = self.add_weight(shape=(self.input_dim, self.output_dim),
                               initializer='random_normal',
                               trainable=True)
    
    def call(self, inputs):
        return tf.linalg.matmul(tf.linalg.matmul(self.a_cap, inputs), self.w)

In [6]:
class GCN_Model(tf.keras.Model):
    def __init__(self, adjacency_matrix, imput_dim, output_dim):
        super(GCN_Model, self).__init__()
        self.layer1 = GCN_Layer(adjacency_matrix, imput_dim, 8*16)
        self.layer2 = GCN_Layer(adjacency_matrix, 8*16, output_dim)

    def call(self, inputs):
        x1 = tf.keras.activations.relu(self.layer1(inputs))
        x2 = self.layer2(x1)
        return x2

In [7]:
import numpy as np
import torch
model = GCN_Model(adj, features.shape[1], tf.unique(labels)[0].shape[0])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss(model, x, y, train_mask, training):
    y_pre = model(x)
    return loss_object(y_true=y[train_mask], y_pred=y_pre[train_mask])

def grad(model, inputs, targets, train_mask):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, train_mask, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

for epoch in range(200):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    loss_value = loss(model, tf.cast(features, dtype=tf.float32), labels, train_mask, training=True)
    loss_value, grads = grad(model, tf.cast(features, dtype=tf.float32), labels, train_mask)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    epoch_loss_avg.update_state(loss_value)
    epoch_accuracy.update_state(labels[val_mask], model(features, training=True)[val_mask])
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 1.790, Accuracy: 23.800%
Epoch 001: Loss: 1.786, Accuracy: 30.800%
Epoch 002: Loss: 1.782, Accuracy: 35.400%
Epoch 003: Loss: 1.778, Accuracy: 38.200%
Epoch 004: Loss: 1.773, Accuracy: 40.800%
Epoch 005: Loss: 1.768, Accuracy: 42.000%
Epoch 006: Loss: 1.763, Accuracy: 43.200%
Epoch 007: Loss: 1.757, Accuracy: 43.400%
Epoch 008: Loss: 1.752, Accuracy: 44.200%
Epoch 009: Loss: 1.745, Accuracy: 44.600%
Epoch 010: Loss: 1.739, Accuracy: 45.000%
Epoch 011: Loss: 1.732, Accuracy: 45.200%
Epoch 012: Loss: 1.724, Accuracy: 45.600%
Epoch 013: Loss: 1.716, Accuracy: 45.800%
Epoch 014: Loss: 1.708, Accuracy: 45.600%
Epoch 015: Loss: 1.700, Accuracy: 46.000%
Epoch 016: Loss: 1.691, Accuracy: 46.200%
Epoch 017: Loss: 1.681, Accuracy: 47.000%
Epoch 018: Loss: 1.671, Accuracy: 47.600%
Epoch 019: Loss: 1.661, Accuracy: 48.200%
Epoch 020: Loss: 1.651, Accuracy: 48.600%
Epoch 021: Loss: 1.640, Accuracy: 49.000%
Epoch 022: Loss: 1.629, Accuracy: 50.000%
Epoch 023: Loss: 1.617, Accuracy: 

Epoch 196: Loss: 0.068, Accuracy: 66.400%
Epoch 197: Loss: 0.067, Accuracy: 66.400%
Epoch 198: Loss: 0.066, Accuracy: 66.400%
Epoch 199: Loss: 0.065, Accuracy: 66.400%


In [13]:
test_accuracy = tf.keras.metrics.Accuracy()
prediction = tf.argmax(model(features, training=False)[test_mask], axis=1, output_type=tf.int64)
test_accuracy(prediction, labels[test_mask])
print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

Test set accuracy: 65.800%
